# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import gmaps
import csv
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities = pd.read_csv("output_data/cities.csv")
cities.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mount Gambier,-37.83,140.77,48.20,100,90,3.36,AU,2020-10-18
1,Te Anau,-45.42,167.72,44.74,92,100,2.19,NZ,2020-10-18
2,Tres Arroyos,-38.37,-60.28,68.81,74,69,6.26,AR,2020-10-18
3,Tasiilaq,65.61,-37.64,33.80,80,58,10.29,GL,2020-10-18
4,Leshukonskoye,64.90,45.76,30.65,97,100,8.19,RU,2020-10-18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

In [6]:
location = cities[["Latitude", "Longitude"]].astype(float)
weights = cities["Humidity"]

layer = gmaps.heatmap_layer(location, weights=weights, max_intensity=100, point_radius=2)

fig.add_layer(layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
criteria = cities.loc[cities["Cloudiness"] <= 10]\
                    .loc[cities["Wind Speed"] < 5]\
                    .loc[cities["Max Temp"] < 80]\
                    .loc[cities["Max Temp"] > 65]

criteria.dropna(inplace=True)

criteria

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,Marzuq,14.40,46.47,68.40,24,0,4.21,YE,2020-10-18
43,Salalah,17.02,54.09,78.80,34,0,4.70,OM,2020-10-18
46,Saint-Philippe,-21.36,55.77,73.40,68,9,3.36,RE,2020-10-18
140,Bima,-8.47,118.72,76.08,87,6,4.25,ID,2020-10-18
173,Hurghada,27.26,33.81,78.80,61,0,4.70,EG,2020-10-18
180,Riyadh,24.69,46.72,68.00,24,0,3.36,SA,2020-10-18
185,Menongue,-14.66,17.69,70.66,25,0,3.85,AO,2020-10-18
201,Kiryat Gat,31.61,34.76,75.99,83,0,3.83,IL,2020-10-18
216,Atambua,-9.11,124.89,72.91,80,9,2.26,ID,2020-10-18
221,Kupang,-10.17,123.58,77.88,72,8,3.98,ID,2020-10-18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_info = pd.DataFrame({"City":criteria["City"], "Country":criteria["Country"], "Lat": criteria["Latitude"],
                         "Lng": criteria["Longitude"]})

hotel_info["Hotel Name"] = ""
hotel_info.head()

,City,Country,Lat,Lng,Hotel Name
36,Marzuq,YE,14.40,46.47,
43,Salalah,OM,17.02,54.09,
46,Saint-Philippe,RE,-21.36,55.77,
140,Bima,ID,-8.47,118.72,
173,Hurghada,EG,27.26,33.81,


In [40]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "lodging","radius": 5000,"key": g_key}
         
for row in criteria.index:
    
    lat, lng = criteria.loc[row, ["Latitude", "Longitude"]]
    
    params["location"] = f"{lat},{lng}"
        
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_info.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("oops")

hotel_info.dropna(inplace=True)
hotel_df = hotel_info.loc[hotel_info["Hotel Name"] != ""]

hotel_df

oops
oops
oops


,City,Country,Lat,Lng,Hotel Name
278,Diré,ML,12.28,-10.97,Lindsey Hôtel


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
for x in range(1,2):
    
    fig = gmaps.figure()
    fig.add_layer(layer)

    markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
    fig.add_layer(markers)
    x += 1
    
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [ ]:
#So, if the person grading this was in our class introduction meeting on 9/1..................this is part of the reason I 
#don't have vacation photos. Evidently I'm very persnickety about the climate where I travel